In [1]:
import requests

def download_and_parse_github_markdown(raw_url):
    response = requests.get(raw_url)
    content = response.text

    lines = content.split('\n')
    questions = []
    current_category = None

    for line in lines:
        line = line.strip()

        if line.startswith("## ") and not line.startswith("###"):
            current_category = line.replace("##", "").strip()

        elif line.startswith("- "):
            question = line[2:].strip()
            questions.append({
                "question": question,
                "category": current_category
            })

    return questions

In [3]:
raw_md_url = "https://raw.githubusercontent.com/zzsza/Datascience-Interview-Questions/refs/heads/master/README.md"
questions = download_and_parse_github_markdown(raw_md_url)
filltered_questions=[]

for q in questions:
  category = q.get("category")

  if category not in ["Contents", "Intro", "Reference"] and category is not None:
    filltered_questions.append(q)

print(len(filltered_questions))
# for i in filltered_questions:
#   print(i)



288


In [4]:
import os
print(os.getcwd())

/home/yuri011228/1-team-YouTIL-ai/Interview/notebook


In [5]:
from transformers import AutoTokenizer, AutoModel
import torch

model_id = "BAAI/bge-m3"

# 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

# cpu로 설정
device = torch.device("cpu")
model = model.to(device)

# 질문 리스트 준비
sentences = [q["question"] for q in filltered_questions]

inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    # 평균 풀링
    embeddings = outputs.last_hidden_state.mean(dim=1)

# 4. 결과 확인
print("임베딩 개수:", len(embeddings))
print("각 벡터 차원:", embeddings[0].shape)

임베딩 개수: 288
각 벡터 차원: torch.Size([1024])


In [8]:
len(embeddings)

288

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

# Qdrant에 연결 (외부 IP 사용)
qdrant = QdrantClient(
    host="34.55.205.42",  # 예: "34.64.12.123"
    port=6333)

collection_name = "interview-questions"

# 벡터 차원 (bge-m3은 1024)
vector_size = embeddings.shape[1]

# 1. 컬렉션 생성 (없다면)
qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
)

# 2. Torch Tensor → list 변환
embedding_list = embeddings.cpu().tolist()

# 3. 업서트할 포인트 준비
points = [
    PointStruct(
        id=i,
        vector=embedding_list[i],
        payload=filltered_questions[i]  # {"question": ..., "category": ...}
    )
    for i in range(len(embedding_list))
]

# 4. 업서트
qdrant.upsert(collection_name=collection_name, points=points)
print("Qdrant 업서트 완료!")


NameError: name 'embeddings' is not defined

In [25]:
qdrant.delete_collection("interview-questions")


True

In [26]:
import os
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModel
import torch
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

# 환경변수 로드
load_dotenv()
qdrant_host = os.getenv("QDRANT_HOST")
qdrant_port = int(os.getenv("QDRANT_PORT"))

# 모델 로드
model_id = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

device = torch.device("cpu")
model = model.to(device)

# Qdrant 연결
qdrant = QdrantClient(host=qdrant_host, port=qdrant_port)
collection_name = "interview-questions"
vector_size = 1024

qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
)

print("모델 및 Qdrant 준비 완료")


모델 및 Qdrant 준비 완료


In [27]:
batch_size = 64  # 또는 32

all_points = []

for i in range(0, len(data_question), batch_size):
    batch = data_question[i:i+batch_size]
    sentences = [item["question"] for item in batch]

    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)

    vectors = embeddings.cpu().tolist()

    for j, vec in enumerate(vectors):
        all_points.append(
            PointStruct(
                id=i + j,
                vector=vec,
                payload=batch[j]
            )
        )

print(f"배치 임베딩 완료. 포인트 수: {len(all_points)}")


배치 임베딩 완료. 포인트 수: 313


In [28]:
qdrant.upsert(collection_name=collection_name, points=all_points)
print(f"Qdrant에 {len(all_points)}개 포인트 업서트 완료!")

Qdrant에 313개 포인트 업서트 완료!


In [19]:
from transformers import AutoTokenizer, AutoModel
import torch
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

load_dotenv()
qdrant_host = os.getenv("QDRANT_HOST")
qdrant_port = int(os.getenv("QDRANT_PORT"))

# 모델 로드
model_id = "BAAI/bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

# CPU 설정
device = torch.device("cpu")

# 질문 리스트 준비
sentences = [q["question"] for q in data_question]

# 토크나이즈
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

# 임베딩 생성 (평균 풀링)
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # (batch_size, 1024)
# 벡터 리스트 변환
embedding_list = embeddings.cpu().tolist()




# Qdrant 포인트 생성
points = [
    PointStruct(
        id=i,
        vector=embedding_list[i],
        payload=data_question[i]  # question, category, level 포함
    )
    for i in range(len(embedding_list))
]

# 업서트 실행
qdrant.upsert(
    collection_name=collection_name,
    points=points
)

print("Qdrant 업서트 완료!")


Qdrant 업서트 완료!


/tmp/ipykernel_2525/223710356.py:43: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(
